![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

Importar librerías

In [48]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Cargar los datos

In [49]:
df = pd.read_csv("rental_info.csv")

Primer vistazo a los datos

In [50]:
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


¿Qué tipo de dato contiene cada columna?

In [51]:
df.dtypes

rental_date          object
return_date          object
amount              float64
release_year        float64
rental_rate         float64
length              float64
replacement_cost    float64
special_features     object
NC-17                 int64
PG                    int64
PG-13                 int64
R                     int64
amount_2            float64
length_2            float64
rental_rate_2       float64
dtype: object

## Preprocesamiento 

#### Crear columna "rental_length_days"

Para crear la columna "rental_length_days" vamos a realizar una operación con "rental_date" y "return_date". Para facilitar eso, primero convertimos estas últimas dos columnas a datetime. 

In [52]:
df[["rental_date", "return_date"]] = df[["rental_date", "return_date"]].apply(pd.to_datetime)

Corroboramos el cambio del tipo de variable

In [53]:
df.dtypes

rental_date         datetime64[ns, UTC]
return_date         datetime64[ns, UTC]
amount                          float64
release_year                    float64
rental_rate                     float64
length                          float64
replacement_cost                float64
special_features                 object
NC-17                             int64
PG                                int64
PG-13                             int64
R                                 int64
amount_2                        float64
length_2                        float64
rental_rate_2                   float64
dtype: object

Creamos la columna "rental_length_days"

In [54]:
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

Nuevamente, inspeccionamos nuestros datos

In [55]:
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


#### Crear dummy variables "deleted_scenes" y "behind_the_scenes" a partir de la columna "special_features"

Inspeccionamos los valores únicos de la columna "special_features"

In [56]:
df["special_features"].unique()

array(['{Trailers,"Behind the Scenes"}', '{Trailers}',
       '{Commentaries,"Behind the Scenes"}', '{Trailers,Commentaries}',
       '{"Deleted Scenes","Behind the Scenes"}',
       '{Commentaries,"Deleted Scenes","Behind the Scenes"}',
       '{Trailers,Commentaries,"Deleted Scenes"}',
       '{"Behind the Scenes"}',
       '{Trailers,"Deleted Scenes","Behind the Scenes"}',
       '{Commentaries,"Deleted Scenes"}', '{Commentaries}',
       '{Trailers,Commentaries,"Behind the Scenes"}',
       '{Trailers,"Deleted Scenes"}', '{"Deleted Scenes"}',
       '{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}'],
      dtype=object)

Creamos las nuevas dummy variables a partir de evaluar si la columna "special_features" contiene o no las frase "Deleted Scenes" y "Behind the Scenes"

In [57]:
# Deleted scenes
df['deleted_scenes'] = df['special_features'].str.contains('Deleted Scenes').astype(int)

# Behind the scenes
df['behind_the_scenes'] = df['special_features'].str.contains('Behind the Scenes').astype(int)

Inspeccionamos las nuevas dummy variables

In [58]:
df.sample(n=5)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scenes
11092,2005-08-20 00:03:18+00:00,2005-08-25 00:09:18+00:00,0.99,2004.0,0.99,181.0,22.99,"{""Behind the Scenes""}",0,0,0,1,0.9801,32761.0,0.9801,5,0,1
14759,2005-07-12 09:02:01+00:00,2005-07-20 10:19:01+00:00,7.99,2009.0,2.99,135.0,17.99,"{Trailers,Commentaries}",0,1,0,0,63.8401,18225.0,8.9401,8,0,0
8729,2005-07-10 00:01:58+00:00,2005-07-10 18:07:58+00:00,4.99,2007.0,4.99,59.0,24.99,"{Trailers,""Deleted Scenes""}",0,0,1,0,24.9001,3481.0,24.9001,0,1,0
3013,2005-07-30 04:15:09+00:00,2005-08-03 03:56:09+00:00,0.99,2009.0,0.99,174.0,21.99,{Commentaries},0,0,0,1,0.9801,30276.0,0.9801,3,0,0
5403,2005-05-27 05:29:31+00:00,2005-06-01 06:27:31+00:00,2.99,2004.0,2.99,159.0,22.99,"{""Behind the Scenes""}",0,0,0,0,8.9401,25281.0,8.9401,5,0,1


#### Crear DataFrame "X" con las variables predictoras 

In [59]:
X = df.drop(["rental_date", "return_date", "special_features", "rental_length_days"], axis=1)

Inspeccionamos el DataFrame "X" de entrenamiento

In [60]:
X.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1


#### Crear Series "y" con la variable objetivo 

In [61]:
y = df["rental_length_days"]

Inspeccionamos la target Series "y"

In [62]:
y.head()

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64

## Entrenamiento de modelo predictor 

Separamos nuestros datos en un conjunto de entrenamiento y uno de prueba

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

Entrenamos distintos modelos y evaluamos cuál es el mejor en términos de MSE

In [64]:
# Definir los modelos a probar
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor()
}

# Entrenar y evaluar cada modelo
for name, model in models.items():
    # Entrenar el modelo
    model.fit(X_train, y_train)
    
    # Predicciones en los datos de prueba
    y_pred = model.predict(X_test)
    
    # Calcular MSE
    mse = mean_squared_error(y_test, y_pred)
    
    # Imprimir el MSE de cada modelo
    print(f"{name}: Mean Squared Error = {mse}")

Linear Regression: Mean Squared Error = 2.9417238646975883
Decision Tree: Mean Squared Error = 2.1643140759368746
Random Forest: Mean Squared Error = 2.0283546335590046


Notamos que el mejor modelo, en términos de MSE, es Random Forest. Procedemos a almacenarlo en la variable "best_model" y almacenar su MSE en la variable "best_mse"

In [65]:
best_model = RandomForestRegressor()      # Inicializar el modelo
best_model.fit(X_train, y_train)          # Entrenar el modelo
y_pred = model.predict(X_test)            # Predicciones en datos de prueba
best_mse = mean_squared_error(y_test, y_pred)  # Cálculo de MSE

print("Best model: Random Forest")
print(f"Best MSE: {best_mse}")

Best model: Random Forest
Best MSE: 2.0283546335590046
